# Regression Problem - Predicting the Apparent Temperature

*data leakage is going to be prevented by using a large amount of data (weather reports) in the train and test sets

our dummy model - ?

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline


In [ ]:
# read the dataset 
weather =pd.read_csv("weatherHistory[1].csv")

In [ ]:
weather

In [ ]:
weather 

In [ ]:
#change misprint
weather=weather.rename(columns={'Loud Cover': 'Cloud Cover'})

# 1. getting to know the data

In [ ]:
weather.describe()

### the numeric variables histogramas:

In [ ]:
atttibutes_hist = weather[["Temperature (C)", "Apparent Temperature (C)", "Wind Speed (km/h)", "Pressure (millibars)",'Cloud Cover']].hist(bins=50, figsize=(20,15))
atttibutes_hist

as we can see, cloud cover has only values of 0 , so we should delete it 

In [ ]:
del weather['Cloud Cover']

### What are the features?
*Formatted Date - includes day, week , month, year

*Summary - sum up of the weather

*Precip Type - which type of raindrops 

*Temperature (C)

*Humidity - Humidity depends on the temperature and pressure .

*Wind Speed (km/h)	

*Wind Bearing (degrees)	- direction of the wind

*Visibility (km) - visibility is a measure of the distance at which an object or light can be clearly discerned.	

*Cloud Cover	(loud cover- misprint)

*Pressure (millibars) - High pressure means the air is heavy . Under high pressure you can generally expect sunny skies and calm weather , whereas Low pressure systems lead to active weather like wind and rain, and also severe weather.

*Daily Summary - daily sum up of the weather


## What is the target variable?

Apparent Temperature : in simple words, Apparent Temperature is the temperature humans percieve, mostly outdoors, as a combination or the actual air temperature, humidity and wind. 

What else do we know?


*we are starting with 12 variables (including the dependent)



<!-- is The perceived temperature in degrees Fahrenheit derived from either a combination of temperature and wind (Wind Chill) or temperature and humidity (Heat Index) for the indicated hour. -
->




<!-- 1. Because the response variable is continuous, this is a regression problem.
2.There are 96453 observations (represented by the rows), and each observation is a weather report from a different date.
 -->

# 2. preprocessing of the data

In [ ]:
weather.dropna()

In [ ]:
weather.isnull().any()

as we can see, there is still nan values in the Precip Type column.

we will handle this by fill in the most common catagory of this feature (rain) instead of the nan values 

In [ ]:
print(weather['Precip Type'].value_counts())

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
weather['Precip Type'] = weather['Precip Type'].fillna("rain")


#### Precip Type, Temperature, wind speed , Wind Bearing, pressure are features that supposed to be  linked very strongly with the Apparent Temperature (according to the information we read), they are crucial for the prediction of the Apparent Temperature. 

on the other hand, daily summary is not that beneficial,  since we have the column Precip Type and also Summary.

*The term "wind direction" is defined as the compass heading FROM which the wind is blowing

In [ ]:
del weather['Daily Summary']

In [ ]:
weather

now , we will split the 'Formatted Date' feature to 4 different numeric attributes in order to get a better understanding of the data  

In [ ]:
import datetime

In [ ]:
weather['Formatted Date'] = pd.to_datetime(weather['Formatted Date'],utc=True)

In [ ]:
weather['year'] = weather['Formatted Date'].dt.year
weather['month'] = weather['Formatted Date'].dt.month
weather['day'] = weather['Formatted Date'].dt.day
weather['weekday'] = weather['Formatted Date'].dt.weekday

In [ ]:
del weather['Formatted Date']

In [ ]:
weather

# checking the relations between humidity and pressure, according to information we've read

As humidity increases pressure decreases

In [ ]:
weather[['Apparent Temperature (C)','Humidity','Visibility (km)','Wind Speed (km/h)']]

getting rid of outliers:


In [ ]:
print(weather['Summary'].value_counts())

In [ ]:
weather = weather[(weather['Summary'] != "Dangerously Windy and Partly Cloudy") &( weather['Summary'] != "Breezy and Dry") & (weather['Summary']!="Windy and Dry") ]
print(weather['Summary'].value_counts())

In [ ]:
weather['Summary']=='Foggy'

### replacing a feature with a better one

In [ ]:
atttibutes_hist = weather[["Temperature (C)","month",'Humidity']].hist(bins=50, figsize=(20,15))
atttibutes_hist

In [ ]:
sns.pairplot(weather, x_vars=['month'], y_vars='Apparent Temperature (C)', height=7, aspect=0.7, kind='reg')

In [ ]:
plt.scatter(weather['Humidity'], weather['Visibility (km)'])

In [ ]:
import seaborn as sns; sns.set_theme(color_codes=True)
ax = sns.regplot(x='Humidity', y="Pressure (millibars)", data=weather)

In [ ]:
'Pressure (millibars)'

In [ ]:
#plt.plot(weather['Pressure (millibars)'], weather['Apparent Temperature (C)'])#, 'b') # 'b' is for blue color -- Shift+Tab for more options
# We can add labels and title
# plt.xlabel('X Axis Title')
# plt.ylabel('Y Axis Title')
# plt.title('Figure/plot Title')
#plt.scatter(weather['Pressure (millibars)'], weather['Humidity'])
sns.pairplot(weather, x_vars='Wind Speed (km/h)', y_vars='Visibility (km)', height=7, aspect=0.7, kind='reg')

conclusion: when humidity is low, pressure is high

#### as we can see above, the Apparent Temperature  is higher in the months that are in the middle of the year.

####  so, we can create a feature that checks if the month is in the middle of the year , or the beggining or the end - we will set the beggining of the middle to be the 5th month, and the end of the middle to be the 9th month

#### this feature will be called: "middle of year" and will have yes/no values.

#### this is a good idea to replace this feature with the "month" feature , since it's giving us a more intuitive relation with the Apparent Temperature, and it probably has a  stronger correlation to it.

In [ ]:
weather['middle of year']=(weather['month']>5) & (weather['month']<9)

In [ ]:
sns.pairplot(weather, x_vars=['middle of year'], y_vars='Apparent Temperature (C)', height=7, aspect=0.7, kind='reg')

as we can see in the correlation below:

In [ ]:
corr_matrix = weather.corr()
corr_matrix['Apparent Temperature (C)'].sort_values(ascending=False)

## 1.2 encoding catagorial data

we need to encode Summary, Precip Type and middle of year to numeric columns

In [ ]:
# i will save the original dataFrame for a later use, for example , for the data visualization.
weather_copy=weather.copy()

In [ ]:
weather['Precip Type'].unique()

In [ ]:
print(weather['middle of year'].value_counts())

In [ ]:
# or 
print(weather['Precip Type'].value_counts())

If a categorical column has just two categories (it's called a binary category), then we can replace their values with 0 and 1. and because the 'Precip Type' column has only 2 catagoreis:

In [ ]:
precip_types = {'rain':0,'snow':1}
weather['precip_type']= weather['Precip Type'].map(precip_types)

In [ ]:
weather['precip_type']

In [ ]:
weather['Summary'].unique()

In contrast to the 'precip_type' column , the Summary column has a lot of catagories, so we will use labelEncoder in order to transform the non-numerical labels to numerical labels  


In [ ]:
from sklearn import preprocessing
lbl_encoder=preprocessing.LabelEncoder()
weather['summary'] = lbl_encoder.fit_transform(weather['Summary'])
weather['summary'].unique()

now, after we encoded these values, we need to handle another problem:
the machine learning model may assume that there is some correlation between these variables, which will produce the wrong output. So to remove this issue, we will use dummy encoding.

For Dummy Encoding, we will use OneHotEncoder class of preprocessing library.

In [ ]:
# #for Country Variable  
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder  
# label_encoder_x= LabelEncoder()  
# weather['Precip Type']= label_encoder_x.fit_transform(weather['Precip Type'])  
# #Encoding for dummy variables  
# onehot_encoder= OneHotEncoder(categories=weather['Precip Type'])    


# pre= onehot_encoder.fit_transform(pre).toarray()  

In [ ]:
# dummy encoding of categorical features
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)


In [ ]:
ohe.fit_transform(weather[['summary']])

In [ ]:
ohe.categories_

In [ ]:
ohe.fit_transform(weather[['precip_type']])

In [ ]:
weather['precip_type']

In [ ]:
ohe.fit_transform(weather[['middle of year']])

In [ ]:
weather['middle of year']=weather['middle of year'].astype(int)

In [ ]:
(weather[['precip_type']]==1.).count()

In [ ]:
weather

## 1.3 doing onehotencoding at the same time on both of the categorial columns


In [ ]:
# X= weather_copy.loc[:, ['Summary','middle of year'] ]

In [ ]:
# X

In [ ]:
weather['middle of year']=pd.get_dummies(weather['middle of year'])

In [ ]:
# use when different features need different preprocessing
from sklearn.compose import make_column_transformer



In [ ]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Summary','middle of year']),
    remainder='passthrough')

In [ ]:
# X=column_trans.fit_transform(X)

In [ ]:
X

now, we will remove Summary and Precip Type columns, cause we alredy made the numeric version of them(summary,precip_type)

In [ ]:
del weather['Summary']
del weather['Precip Type']

In [ ]:
weather

In [ ]:
pd.DataFrame(data={'features': weather.columns})

In [ ]:
weather

# 3. data visualization

 using the preproccessed dataFrame, before scaling, 
in order to get meaningful and clear picture of the data

In [ ]:
# import matplotlib.pyplot as plt
# weather.plot(x='Apparent Temperature (C)', y=["precip_type"])
# plt.show()

In [ ]:
AT_column = weather['Apparent Temperature (C)']

In [ ]:
AT_column.plot(kind="hist")
AxesSubplot='Frequency'

##### this means that the most frequent Apparent Temperature is in the range between 10 to 20 celcius

In [ ]:
# import matplotlib
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.express as px
# %matplotlib inline


In [ ]:
# import matplotlib
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.express as px
# %matplotlib inline

# fig , axs = plt.subplots(ncols=7,nrows=2, figsize=(20,10))
# index=0
# axs= axs.flatten()
# for k ,v in weather.items():
#     sns.boxplot(y=k, data=weather , ax=axs[index])
#     index +=1
# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
# fig , axs = plt.subplots(ncols=7,nrows=2, figsize=(20,10))
# index=0
# axs= axs.flatten()
# for k ,v in weather.items():
#     sns.boxplot(y=k, data=weather , ax=axs[index])
#     index +=1
# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

# correlations

### correlation matrix

In [ ]:
#weather['pressure-humidity-ratio']=weather['Humidity']/weather['Pressure (millibars)']

In [ ]:
#weather['pressure-humidity-ratio']

In [ ]:
#weather['humidity-wind-ratio']=weather['Wind Speed (km/h)']/weather['Humidity']

In [ ]:
weather['wind-visibility-ratio']=weather['Wind Speed (km/h)']/weather['Visibility (km)']

In [ ]:
#weather.describe()[['humidity-wind-ratio', 'wind-visibility-ratio','Visibility (km)']]

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(weather.corr(),annot=True);

as we can see from the heatmap above, the correlation between the Apparent Temperature to the pressure is very low

In [ ]:
# del weather['Pressure (millibars)']
del weather['weekday']
del weather['day']
del weather['year']

In [ ]:
# numerical  features correlation:

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.pairplot(weather, x_vars=['Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)','Visibility (km)'], y_vars='Apparent Temperature (C)', height=7, aspect=0.7, kind='reg')

# categorial features correlation:

In [ ]:
weather_copy['middle of year']

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 15))
fig.suptitle('categorial features:')

sns.swarmplot(ax=axes[0,0], x="Temperature (C)", y="Apparent Temperature (C)", data=weather_copy)
sns.swarmplot(ax=axes[0,1], x="Humidity", y="Apparent Temperature (C)", data=weather_copy)
sns.swarmplot(ax=axes[1,0], x="Wind Speed (km/h)", y="Apparent Temperature (C)", data=weather_copy)
sns.swarmplot(ax=axes[1,1], x="Wind Bearing (degrees)", y="Apparent Temperature (C)s", data=weather_copy)

# 4. choosing a regression metric -

### We chose Mean abs error as a metric due to the explanations below and also beacause of the following dummy model results   (mae had a better result than rmse)

##### Mean Squared Error (MSE)

The MSE is calculated as the mean or average of the squared differences between predicted and expected target values in a dataset

when MSE is used as a loss function (regression metric), a lot of weight is given to larger errors because of the square
The units of the MSE are squared units.

A model that achieves an MSE better than the MSE for the naive model has skill.

In [ ]:
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error

In [ ]:
# example of calculate the mean squared error
# real value
expected = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
# predicted value
predicted = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]
# calculate errors
errors = mean_squared_error(expected, predicted)
# report error
print(errors)

##### Root Mean Squared Error

the units of the RMSE are the same as the original units of the target value that is being predicted, in contrast to MSE.
this is the advantage comapring to MSE.

In [ ]:
mean_squared_error(y_true, y_pred, squared=False)

#### Mean abs error

like RMSE, the units of the error score match the units of the target value that is being predicted.

Unlike the RMSE, the changes in MAE are linear and therefore intuitive.

In [ ]:
# example of calculate the mean absolute error
from sklearn.metrics import mean_absolute_error
# real value
expected = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
# predicted value
predicted = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]
# calculate errors
errors = mean_absolute_error(expected, predicted)
# report error
print(errors)

*The lower value of MAE, MSE, and RMSE implies higher accuracy of a regression model. However, a higher value of R square is considered desirable.

In [ ]:
R Square

R Squared & Adjusted R Squared are used for explaining how well the independent variables in the linear regression model explains the variability in the dependent variable.

# 5. Split the data to train and test

In [ ]:
X = weather.drop(['Apparent Temperature (C)'],axis=1)
y = weather['Apparent Temperature (C)']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [ ]:
#train=X_train

In [ ]:
X_train

In [ ]:
X_test.shape

In [ ]:
y_train

## Scaling


את הנירמול לא נעשה על משתנים קטגרויאליים ושמיים שעשינו עליהם כבר encoding  
,כי אין אפשרות להשוות
ניצור גאטא פריים חדש מנורמל
את הויזואליצזיה של הדאטא נעשה על הדאטא פריים המקורי, הלא מנורמל, כדי שנראה את המידע בצורה ברורה ונראה מה המספרים מייצגים

we will do the scaling only on numeric variables i.e not on the categorial . 
we will creaze a normalized dataFrame 

In [ ]:
X_train.head()
# delete presure , day month weekday ...
#percip type and summary are catagorials

*we are scaling only the numrical features, i.e , the features we didn't encoded

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler_standard = StandardScaler()
scaler_MinMax = MinMaxScaler()

In [ ]:
X_train_standardized = X_train.copy()
X_test_standardized = X_test.copy()
X_train_normalized = X_train.copy()
X_test_normalized = X_test.copy()

In [ ]:
numerical_features = ["Temperature (C)","Humidity","Wind Speed (km/h)","Visibility (km)","month"]

In [ ]:
# Standardization:
scaler_standard.fit(X_train[numerical_features])
X_train_standardized[numerical_features] = scaler_standard.transform(X_train_standardized[numerical_features])

# the scaling is with the the same fitted scaler (by the train data)
#only transform on the test data
X_test_standardized[numerical_features] = scaler_standard.transform(X_test_standardized[numerical_features])

In [ ]:
print('note: the mean is 0 and std is 1')
X_train_standardized.describe()[numerical_features].iloc[[1, 2]]

In [ ]:
# Normalization:
scaler_MinMax.fit(X_train[numerical_features])
X_train_normalized[numerical_features] = scaler_MinMax.transform(X_train_normalized[numerical_features])

# the scaling is with the the same fitted scaler (by the train data)
#only transform on the test data
X_test_normalized[numerical_features] = scaler_MinMax.transform(X_test_normalized[numerical_features])

In [ ]:
print('note: the mean is 0 and std is 1')
X_train_standardized.describe()[numerical_features].iloc[[1, 2]]

In [ ]:
X_train_scale=X_train.copy()
y_train_scale=y_train.copy()

deleting the encoded variables (categorials)

In [ ]:
# del X_train_scale['summary']
# del X_train_scale['precip_type']

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

In [ ]:
# scaling= MinMaxScaler()

*on the train set -fit and transform 

In [ ]:
#scaling.fit_transform(X_train_scale)

In [ ]:
#arr_y_train=np.array(y_train_scale)

In [ ]:
#scaling.fit_transform(arr_y_train.reshape(-1,1))

In [ ]:
#arr_x_test=np.array(X_train_scale)

*on the test set -transform only 

In [ ]:
#scaling.transform(arr_x_test.reshape(-1,1))

In [ ]:
#arr_y_test=np.array(y_train_scale)

In [ ]:
#scaling.transform(arr_y_test.reshape(-1,1))

## dealing with imbalanced data

*** in this part we do not touch the test set!
 
 
 we will only use the test set for the dummy model , and after the cross validation

the data is imbalanced as we can see from the histogram below

In [ ]:
X_train

In [ ]:
plt.hist(weather, range=(1,100))

In [ ]:
# Histogram 
# from random import sample
# data = sample(range(1, 1000), 100)
plt.hist(train) # see if i can change 

In [ ]:
# Histogram 
plt.hist(X_train)

In [ ]:
X_train_scale= X_train_standardized

In [ ]:
y_train_scale = y_train

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from collections import Counter
# from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where

# summarize class distribution
counter = Counter(y_train)
print(counter)

oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)


counter = Counter(y_train)
print(counter)

# scatter plot of examples by class label
for label, _ in counter.items():
    row_ix = where(y_train == label)[0]
    pyplot.scatter(X_train[row_ix, 0], X_train[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

## dummy model

In [ ]:
from sklearn.dummy import DummyRegressor

from sklearn.metrics import mean_absolute_error as MAE



In [ ]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)
R2_score = dummy_regr.score(X_test, y_test)
y_predict = dummy_regr.predict(X_test)
mae = MAE(y_test, y_predict)
rmse= mean_squared_error(y_test, y_predict, squared=False)


print ('The dummy model have a root mean squared error of '+ str(rmse) )

# errorsRMSE

In [ ]:
print('The dummy model have a R2 score of ' + str(R2_score)[:6] + " as expected (around 0), and mean absolute error of " + str(mae)[:4])

# The dummy model have a R2 score of -7.151 as expected (around 0), and mean absolute error of 9.02

In [ ]:
# dummy_regr = DummyRegressor(strategy="mean")
# dummy_regr.fit(X_train, y_train)
# dummy_regr.predict(X_train, y_train)
# dummy_regr.score(X_train, y_train)

my dummy model

the mean of the Apparent Temperatures

In [ ]:
mean=weather['Apparent Temperature (C)'].mean()

# comparing different Regression Models

*cross validation allows us to compare different machine learning methods and get a sense of how well they will work in practice. the advantage here, is that it uses different blocks of data for train and test, and by that, "prepare" the model in the best way to predict outcome for new data  

 # Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

In [ ]:
LR = LinearRegression()

In [ ]:
y_train

In [ ]:
kf = KFold(n_splits=10, random_state=42, shuffle=True)

R2_scores_standardized = cross_val_score(LR, X_train_standardized, y_train, cv=kf)
y_predict_standardized = cross_val_predict(LR, X_train_standardized, y_train, cv=kf)
mae_standarsized = MAE(y_train, y_predict_standardized)

R2_scores_normalized = cross_val_score(LR, X_train_normalized, y_train, cv=kf)
y_predict_normalized = cross_val_predict(LR, X_train_normalized, y_train, cv=kf)
mae_normalized = MAE(y_train, y_predict_normalized)

In [ ]:
fig, axes = plt.subplots(1,2)
((ax1, ax2)) = axes

y_predicted = cross_val_predict(LR, X_train_standardized, y_train, cv=kf)
ax1.scatter(y_train, y_predicted, alpha=0.3, color='orange')
ax1.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=4)
ax1.set_xlabel('Actual')
ax1.set_ylabel('Predicted')
ax1.set_title('standardized:')

y_predicted = cross_val_predict(LR, X_train_normalized, y_train, cv=kf)
ax2.scatter(y_train, y_predicted, alpha=0.3, color='red')
ax2.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=4)
ax2.set_xlabel('Actual')
ax2.set_ylabel('Predicted')
ax2.set_title('normalized:')

plt.show()

In [ ]:
#standadized train set cross validation:

In [ ]:
print("the scores of cross validation are:")
print(R2_scores_standardized)
print()
print("mean R2 is: " + str(R2_scores_standardized.mean())[:5] + " with std of  " + str(R2_scores_standardized.std())[:5] + " and MAE of " + str(mae_standarsized)[:6])

In [ ]:
#normalized train set cross validation:

In [ ]:
print("the scores of cross validation are:")
print(R2_scores_normalized)
print()
print("mean R2 is: " + str(R2_scores_normalized.mean())[:5] + " with std of  " + str(R2_scores_normalized.std())[:5] + " and MAE of " + str(mae_normalized)[:6])

although the results are the same with normalized and standardized,
we can see below that the feature values are definetly different 

In [ ]:
X_train_normalized

In [ ]:
X_train_standardized

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [ ]:
RFR = RandomForestRegressor()

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, y_train)

In [ ]:
kf = KFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
R2_scores_standardized = cross_val_score(regr, X_train_standardized, y_train, cv=kf)
y_predict_standardized = cross_val_predict(regr, X_train_standardized, y_train, cv=kf)
mae_standarsized = MAE(y_train, y_predict_standardized)

In [ ]:
R2_scores_normalized = cross_val_score(regr, X_train_normalized, y_train, cv=kf)
y_predict_normalized = cross_val_predict(regr, X_train_normalized, y_train, cv=kf)
mae_normalized = MAE(y_train, y_predict_normalized)

In [ ]:
print("the scores of cross validation are:")
print(R2_scores_standardized)
print()
print("mean R2 is: " + str(R2_scores_standardized.mean())[:5] + " with std of  " + str(R2_scores_standardized.std())[:5] + " and MAE of " + str(mae_standarsized)[:6])

# Keras Regressions

In [ ]:
#! pip install keras

In [ ]:
#! pip install tensorflow

In [ ]:
X_train


In [ ]:

import tensorflow as tf

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
# Also, ADAM optimization algorithm is used for optimizing loss function (Mean squared error)

In [ ]:
keras.metrics.RootMeanSquaredError()

In [ ]:
model = Sequential()
model.add(Dense(11,activation='relu'))
model.add(Dense(11,activation='relu'))
model.add(Dense(11,activation='relu'))
model.add(Dense(11,activation='relu'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae',tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
y_test

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
X_test= np.asarray(X_test).astype(np.float32)
y_test= np.asarray(y_test).astype(np.float32)

In [ ]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=128,epochs=400)
model.summary()

In [ ]:
loss_df = pd.DataFrame(model.history.history)
loss_df.plot(figsize=(12,8))

In [ ]:
model.history.history['val_mse']

In [ ]:
model.history.history['val_mae']

In [ ]:
model.history.history['val_root_mean_squared_error']

In [ ]:
# plot metrics
pyplot.plot(loss_df ['mse'])


In [ ]:
pyplot.plot(loss_df ['mae'])


In [ ]:
pyplot.plot(loss_df ['root_mean_squared_error'])

In [ ]:

print(loss_df.keys())

# hyperparameter tuning 
I will try to increase the R2 score and the MAE of the Random Forest Regressor model by choosing the best hyperparams.

## random search 

With small data sets and lots of resources, Grid Search will produce accurate results. However, with large data sets, the high dimensions will greatly slow down computation time and be very costly. Therefore, we chose to use random search for 

random search requires two arguments. The first is the model that you are optimizing. This is an instance of the model with values of hyperparameters set that you want to optimize. 
The second is the search space:
This is defined as a dictionary where the names are the hyperparameter arguments to the model and the values are discrete values or a distribution of values to sample in the case of a random search.

In [946]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from scipy.stats import loguniform

In [947]:
cv=RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [948]:
# define model
model = Ridge()
# define search space
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = loguniform(1e-5, 100)
space['fit_intercept'] = [True, False]
space['normalize'] = [True, False]

# define search
# pattern - search = GridSearchCV(model, space)
search = RandomizedSearchCV(model, space, n_iter=500, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv, random_state=1)

In [950]:
# execute search
result = search.fit(X_train_normalized, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

KeyboardInterrupt: 